In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter

from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./data/data_after_aliona_pipline.csv')
df['set_type'] = df['set_type'].astype('string')
df = df.select_dtypes(exclude=['object'])
df = df.set_index('ID')

In [3]:
# Для обучения и предсказание на тесте 
X_train_val = df[df['set_type']!='test']
y_train_val = X_train_val['Price']
X_train_val = X_train_val.drop(['set_type','Price'], axis=1)

train = df[df['set_type']=='train']
val = df[df['set_type']=='val']
test = df[df['set_type']=='test']

train = train.drop('set_type', axis=1)
val = val.drop('set_type', axis=1)
test = test.drop('set_type', axis=1)

y_train = train['Price']
X_train = train.drop('Price', axis=1)

y_val = val['Price']
X_val= val.drop('Price', axis=1)

y_test = test['Price']
X_test = test.drop('Price', axis=1)

plot_df = X_val[['Количество комнат', 'Площадь общая', 'Год постройки']]
plot_df = pd.merge(plot_df, y_val, left_index=True, right_index=True)

X_val

,Количество комнат,Раздельных комнат,Площадь общая,Площадь жилая,Площадь кухни,Год постройки,Новостройка,Ремонт,Стоянка автомобиля,Площадь по СНБ,Высота потолков,Площадь балконов,Мебель,Видео-домофон,Телефон,Огороженная территория,Гараж,Сигнализация,Год капитального ремонта,Жилой комплекс,Тип населенного пункта,Этаж квартиры,Всего этажей,Продавец,Широта,Долгота,Дистанция до обл. центра,Дистанция до центра Минска,Наличие балкона,Количество балконов,Количество застекленных балконов,Наличие лоджии,Количество лоджий,Количество застекленных лоджий,Наличие террасы,Наличие веранды,Наличие_эркера,not_finish,just_finish,has_renovation,condition_dreadful,time_after_renovation,first_floor,last_floor,Планировка_unknow,Планировка_Брежневка,Планировка_Малосемейка,Планировка_Пентхаус,Планировка_Свободная планировка,Планировка_Сталинка,Планировка_Стандартный проект,Планировка_Студия,Планировка_Таунхаус,Планировка_Улучшенный проект,Планировка_Хрущевка,Планировка_Чешский проект,Тип дома_unknow,Тип дома_Блок-комнаты,Тип дома_Бревенчатый,Тип дома_Каркасно-блочный,Тип дома_Кирпичный,Тип дома_Монолитный,Тип дома_Панельный,Тип дома_Силикатные блоки,Санузел_2 и более,Санузел_3 сан.узла,Санузел_4 сан.узла,Санузел_unknow,Санузел_Раздельный,Санузел_Совмещенный,Полы_unknow,Полы_Бетонная стяжка,Полы_ДСП,Полы_Деревянные,Полы_Ковровое покрытие,Полы_Ламинированные,Полы_Линолеум,Полы_Паркет,Полы_Плитка,Вид этажа_usual_floor,Вид этажа_Первый-высокий,Вид этажа_Первый-низкий,Вид этажа_Технический этаж,Число уровней_1.0,Число уровней_2.0,Число уровней_3.0,Область_Брестская,Область_Витебская,Область_Гомельская,Область_Гродненская,Область_Минск,Область_Минская,Область_Могилевская,Район_Барановичский,Район_Белыничский,Район_Березинский,Район_Березовский,Район_Берестовицкий,Район_Бешенковичский,Район_Бобруйский,Район_Борисовский,Район_Брагинский,Район_Браславский,Район_Брестский,Район_Буда-Кошелевский,Район_Быховский,Район_Верхнедвинский,Район_Ветковский,Район_Вилейский,Район_Витебский,Район_Волковысский,Район_Воложинский,Район_Ганцевичский,Район_Глубокский,Район_Глусский,Район_Гомельский,Район_Горецкий,Район_Городокский,Район_Гродненский,Район_Дзержинский,Район_Добрушский,Район_Докшицкий,Район_Дрибинский,Район_Дрогичинский,Район_Дубровенский,Район_Дятловский,Район_Ельский,Район_Жабинковский,Район_Житковичский,Район_Жлобинский,Район_Зельвенский,Район_Ивановский,Район_Ивацевичский,Район_Ивьевский,Район_Калинковичский,Район_Каменецкий,Район_Кировский,Район_Клецкий,Район_Климовичский,Район_Кличевский,Район_Кобринский,Район_Копыльский,Район_Кореличский,Район_Кормянский,Район_Костюковичский,Район_Краснопольский,Район_Кричевский,Район_Круглянский,Район_Крупский,Район_Лельчицкий,Район_Лепельский,Район_Лидский,Район_Лиозненский,Район_Логойский,Район_Лоевский,Район_Лунинецкий,Район_Любанский,Район_Ляховичский,Район_Малоритский,Район_Минск,Район_Минский,Район_Миорский,Район_Могилевский,Район_Мозырский,Район_Молодечненский,Район_Мостовский,Район_Мстиславский,Район_Мядельский,Район_Наровлянский,Район_Несвижский,Район_Новогрудский,Район_Октябрьский,Район_Оршанский,Район_Осиповичский,Район_Островецкий,Район_Ошмянский,Район_Петриковский,Район_Пинский,Район_Полоцкий,Район_Поставский,Район_Пружанский,Район_Пуховичский,Район_Речицкий,Район_Рогачевский,Район_Россонский,Район_Светлогорский,Район_Свислочский,Район_Сенненский,Район_Слонимский,Район_Слуцкий,Район_Смолевичский,Район_Сморгонский,Район_Солигорский,Район_Стародорожский,Район_Столбцовский,Район_Столинский,Район_Толочинский,Район_Узденский,Район_Ушачский,Район_Хойникский,Район_Хотимский,Район_Чаусский,Район_Чашникский,Район_Червенский,Район_Чериковский,Район_Чечерский,Район_Шарковщинский,Район_Шкловский,Район_Шумилинский,Район_Щучинский,Район города_Missing,Район города_Железнодорожный,Район города_Заводской,Район города_Ленинский,Район города_Московский,Район города_Новобелицкий,Район города_Октябрьский,Район города_Партизанский,Район города_Первомайский,Район города_Советский,Район города_Фрунзенский,Район города_Центральный,Р

# Функция вывода

In [4]:
def tripal_print(pred_train, pred_val):

    def error(y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        rmse = root_mean_squared_error(y_true, y_pred)
        return mse, rmse
    
    print(f'Train:      mse= {error(y_train, pred_train)[0]:.2f}, rmse={error(y_train, pred_train)[1]:.2f}')
    print(f'Validation: mse= {error(y_val, pred_val)[0]:.2f}, rmse={error(y_val, pred_val)[1]:.2f}')

In [5]:
def error0(y_true,y_pred,size_x,sizy_y):

    def millions_formatter(x, pos):
        return '%1.1fтыс.' % (x * 1e-3)

    error = y_true - y_pred
    
    plt.figure(figsize=(size_x, sizy_y))
    plt.scatter(y_pred, error, color='blue', edgecolor='k')
    plt.axhline(y=0, color='red', linestyle='--') # Линия, показывающая отсутствие ошибки
    plt.xlabel('Предсказанные значения')
    plt.ylabel('Остатки')
    plt.title('График остатков')
    plt.gca().yaxis.set_major_formatter(FuncFormatter(millions_formatter))
    plt.gca().xaxis.set_major_formatter(FuncFormatter(millions_formatter))
    plt.show()

In [6]:
def long_pred(y_pred):
    plot_df['pred'] =  y_pred #plot_df['Price'] -
    plot_df.groupby('Площадь общая')['Price'].mean().plot(figsize=(18, 3))
    plot_df.groupby('Площадь общая')['pred'].mean().plot(figsize=(18, 3), title='pred by Площадь общая')
    plt.legend()
    plt.show()
    plot_df.groupby('Количество комнат')['Price'].mean().plot(figsize=(18, 3))
    plot_df.groupby('Количество комнат')['pred'].mean().plot(figsize=(18, 3), title='pred by Количество комнат')
    plt.legend()
    plt.show()
    plot_df.groupby('Год постройки')['Price'].mean().plot(figsize=(18, 3))
    plot_df.groupby('Год постройки')['pred'].mean().plot(figsize=(18, 3), title='pred by Год постройки')
    plt.legend()
    plt.show()

# Baseline

In [7]:
pred_train_mas = np.full(shape=y_train.shape, fill_value=y_train.mean())
pred_val_mas = np.full(shape=y_val.shape, fill_value=y_train.mean())

tripal_print(pred_train_mas, pred_val_mas)

Train:      mse= 2890988907.15, rmse=53767.92
Validation: mse= 3540328053.90, rmse=59500.66


# Linear

In [9]:
scaler = MinMaxScaler(feature_range=(-0.5, 0.5))

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

X_train_val = scaler.fit_transform(X_train_val)

In [10]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
lr_train_y_pred = linear_regressor.predict(X_train)
lr_val_y_pred = linear_regressor.predict(X_val)


In [11]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train_val, y_train_val)
lr_test_y_pred = linear_regressor.predict(X_test)


In [12]:
name_model = 'linear_regressor'
submission_df_val = pd.DataFrame()
submission_df_val['ID'] = y_val.index
submission_df_val[f'pred_val_{name_model}'] = lr_val_y_pred
submission_df_val.to_csv(f'./result_after_aliona_pipline/val/prediction_val_{name_model}.csv',index=False)


submission_df_test = pd.DataFrame()
submission_df_test['ID'] = y_test.index
submission_df_test[f'pred_test_{name_model}'] = lr_test_y_pred
submission_df_test.to_csv(f'./result_after_aliona_pipline/test/prediction_test_{name_model}.csv',index=False)
submission_df_test

,ID,pred_test_linear_regressor
0,3209254,226840.0
1,3228478,176056.0
2,3252199,167088.0
3,2830692,169656.0
4,3189739,176544.0
...,...,...
3158,3254162,46768.0
3159,3212275,38528.0
3160,3115929,170968.0
3161,3212250,59200.0


# L1 regularization (Lasso)

In [19]:
scaler = MinMaxScaler(feature_range=(-0.5, 0.5))

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

X_train_val = scaler.fit_transform(X_train_val)

In [20]:
lasso_regressor = Lasso()
lasso_regressor.fit(X_train, y_train)
Lasso_pred_val = lasso_regressor.predict(X_val)

c:\Users\Asus\.conda\envs\work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.704e+11, tolerance: 3.972e+09
  model = cd_fast.enet_coordinate_descent(


In [21]:
lasso_regressor = Lasso()
lasso_regressor.fit(X_train_val, y_train_val)
Lasso_pred_test = lasso_regressor.predict(X_test)

c:\Users\Asus\.conda\envs\work\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.449e+12, tolerance: 5.468e+09
  model = cd_fast.enet_coordinate_descent(


In [22]:
name_model = 'Lasso_regressor'
submission_df_val = pd.DataFrame()
submission_df_val['ID'] = y_val.index
submission_df_val[f'pred_val_{name_model}'] = Lasso_pred_val
submission_df_val.to_csv(f'./result_after_aliona_pipline/val/prediction_val_{name_model}.csv',index=False)


submission_df_test = pd.DataFrame()
submission_df_test['ID'] = y_test.index
submission_df_test[f'pred_test_{name_model}'] = Lasso_pred_test
submission_df_test.to_csv(f'./result_after_aliona_pipline/test/prediction_test_{name_model}.csv',index=False)
submission_df_test

,ID,pred_test_Lasso_regressor
0,3209254,227926.986981
1,3228478,176829.600236
2,3252199,167806.343435
3,2830692,172060.806409
4,3189739,176829.857274
...,...,...
3158,3254162,49462.571161
3159,3212275,37692.556700
3160,3115929,171698.882129
3161,3212250,61770.833955


# L2 regularization (Ridge)

In [23]:
alpha = 1
Ridge_regressor = Ridge(alpha=alpha)
Ridge_regressor.fit(X_train, y_train)
Ridge_pred_val = Ridge_regressor.predict(X_val)

In [24]:
alpha = 1
Ridge_regressor = Ridge(alpha=alpha)
Ridge_regressor.fit(X_train_val, y_train_val)
Ridge_pred_test = Ridge_regressor.predict(X_test)

In [26]:
name_model = 'Ridge_regressor'
submission_df_val = pd.DataFrame()
submission_df_val['ID'] = y_val.index
submission_df_val[f'pred_val_{name_model}'] = Ridge_pred_val
submission_df_val.to_csv(f'./result_after_aliona_pipline/val/prediction_val_{name_model}.csv',index=False)


submission_df_test = pd.DataFrame()
submission_df_test['ID'] = y_test.index
submission_df_test[f'pred_test_{name_model}'] = Ridge_pred_test
submission_df_test.to_csv(f'./result_after_aliona_pipline/test/prediction_test_{name_model}.csv',index=False)
submission_df_test

,ID,pred_test_Ridge_regressor
0,3209254,229805.271322
1,3228478,178109.515484
2,3252199,167768.497857
3,2830692,172937.375863
4,3189739,175219.271362
...,...,...
3158,3254162,50242.119834
3159,3212275,38044.927954
3160,3115929,170628.486184
3161,3212250,61993.951246


# Polynomial regression

In [27]:
degree = 2
alpha = 1
poly_features = make_pipeline(PolynomialFeatures(degree=degree), Ridge(alpha=alpha))

X_poly_train = poly_features.fit(X_train, y_train)

pl2_pred_val = X_poly_train.predict(X_val)

In [28]:
degree = 2
alpha = 1
poly_features = make_pipeline(PolynomialFeatures(degree=degree), Ridge(alpha=alpha))

X_poly_train = poly_features.fit(X_train_val, y_train_val)

pl2_pred_test= X_poly_train.predict(X_test)

In [29]:
name_model = 'Polynomial_regression'
submission_df_val = pd.DataFrame()
submission_df_val['ID'] = y_val.index
submission_df_val[f'pred_val_{name_model}'] = pl2_pred_val
submission_df_val.to_csv(f'./result_after_aliona_pipline/val/prediction_val_{name_model}.csv',index=False)


submission_df_test = pd.DataFrame()
submission_df_test['ID'] = y_test.index
submission_df_test[f'pred_test_{name_model}'] = pl2_pred_test
submission_df_test.to_csv(f'./result_after_aliona_pipline/test/prediction_test_{name_model}.csv',index=False)
submission_df_test

,ID,pred_test_Polynomial_regression
0,3209254,392274.391980
1,3228478,194479.956642
2,3252199,195568.533928
3,2830692,187338.074096
4,3189739,222534.245208
...,...,...
3158,3254162,38204.227825
3159,3212275,30356.370441
3160,3115929,131621.058359
3161,3212250,44100.334116
